In [10]:
import os
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

In [11]:
col_names = [
    'age', 
    'workclass', 
    'fnlwgt', 
    'education', 
    'education-num', 
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week', 
    'native-country',
    'income']

np.random.seed(0)

In [13]:
DATA_DIR = "data"
fn = os.path.join(DATA_DIR, "adult.csv")
df = pd.read_csv(fn, header=None, sep=";", names=col_names)

In [14]:
numeric_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain','capital-loss', 'hours-per-week']
categorical_cols = [c for c in col_names if c not in numeric_cols]

# here we select which are the input columns and which the output columns
target_col = 'native-country'
source_cols = [c for c in df.columns if c!=target_col]

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, [c for c in numeric_cols if c!=target_col]),
        ('cat', categorical_transformer, [c for c in categorical_cols if c!=target_col])])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(solver='lbfgs'))])

X = df.drop(target_col, axis=1)
y = df[target_col]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

/home/philipp/anaconda3/envs/mlfd/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/philipp/anaconda3/envs/mlfd/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/philipp/anaconda3/envs/mlfd/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


model score: 0.899


/home/philipp/anaconda3/envs/mlfd/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
